In [1]:
import pandas as pd 



In [2]:
data =pd.read_csv('Sheet_1.csv')

In [3]:
data = pd.concat([data["class"],data["response_text"]], axis = 1)
# Sheet_1.drop(["response_id","Unnamed: 3","Unnamed: 4","Unnamed: 5","Unnamed: 6","Unnamed: 7"], axis = 1, inplace = True)

data.dropna(axis = 0, inplace=True)
data

,class,response_text
0,not_flagged,I try and avoid this sort of conflict
1,flagged,Had a friend open up to me about his mental ad...
2,flagged,I saved a girl from suicide once. She was goin...
3,not_flagged,i cant think of one really...i think i may hav...
4,not_flagged,Only really one friend who doesn't fit into th...
...,...,...
75,not_flagged,"Now that I've been through it, although i'm no..."
76,flagged,when my best friends mom past away from od'ing...
77,not_flagged,As a camp counselor I provide stability in kid...
78,flagged,My now girlfriend used to have serious addicti...


In [4]:
data['class'] = data['class'].map({'not_flagged':0, 'flagged':1})

In [5]:
data.isnull().sum()

class            0
response_text    0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   class          80 non-null     int64 
 1   response_text  80 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [7]:
#lets apply on one series aggrgate,tokenization, stopwords, lemmatazation
data['response_text'][4]

'Only really one friend who doesn\'t fit into the any of the above categories. Her therapist calls it spiraling." Anyway she pretty much calls me any time she is frustrated by something with  her boyfriend to ask me if it\'s logical or not. Before they would just fight and he would call her crazy. Now she asks me if it\'s ok he didn\'t say "please" when he said  "hand me the remote."'

In [8]:
import re
first_Res_text = data['response_text'][4]
Res_text = re.sub("[^a-zA-Z]"," ", first_Res_text).lower()
Res_text

'only really one friend who doesn t fit into the any of the above categories  her therapist calls it spiraling   anyway she pretty much calls me any time she is frustrated by something with  her boyfriend to ask me if it s logical or not  before they would just fight and he would call her crazy  now she asks me if it s ok he didn t say  please  when he said   hand me the remote  '

In [9]:
#nltk
import nltk
from nltk.corpus import stopwords

Res_text = nltk.word_tokenize(Res_text)
Res_text = [word for word in Res_text if not word in set(stopwords.words('english'))]


In [10]:
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shedr\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shedr\AppData\Roaming\nltk_data...


True

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
Res_text = [lemma.lemmatize(word) for word in Res_text]
Res_text = " ".join(Res_text)

In [12]:
import nltk as nlp
description_list = [] # List
for description in data.response_text:
       
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower() 
    
    description = nltk.word_tokenize(description)
    description = [ word for word in description if not word in set(stopwords.words("english"))]
    
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    
    description = " ".join(description)
    description_list.append(description)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
max_feature = 500

cv = CountVectorizer(max_features = max_feature, stop_words = "english")

space_matrix = cv.fit_transform(description_list).toarray() # x

print("Most frequently used {} words {}".format(max_feature, cv.get_feature_names_out()))

Most frequently used 500 words ['able' 'absolutely' 'acquaintance' 'acted' 'action' 'activity'
 'addiction' 'adequate' 'admit' 'advice' 'advise' 'age' 'ago' 'agony'
 'alcoholic' 'allowed' 'anniversary' 'answer' 'anxiety' 'anxious'
 'apposed' 'ask' 'asks' 'attention' 'aunt' 'avoid' 'away' 'bad'
 'basically' 'bedroom' 'best' 'better' 'big' 'bit' 'blow' 'blue' 'blunt'
 'book' 'boyfriend' 'break' 'bring' 'brother' 'brought' 'bunch' 'called'
 'calling' 'calm' 'came' 'camp' 'camping' 'campsite' 'cancer' 'car' 'care'
 'caring' 'category' 'caught' 'cause' 'chance' 'change' 'changed' 'chat'
 'circumstance' 'clean' 'cleaning' 'cocaine' 'come' 'comfort' 'comforted'
 'commit' 'common' 'complete' 'completely' 'concern' 'confines' 'conflict'
 'convinced' 'cop' 'cope' 'counselor' 'countless' 'couple' 'crazy'
 'cutter' 'cutting' 'damn' 'dating' 'day' 'dealing' 'dealt' 'death'
 'defined' 'depressed' 'depression' 'describes' 'desire' 'diagnosed'
 'dialog' 'difficulty' 'disorder' 'doc' 'dont' 'douche' 'd

In [14]:
x = space_matrix
y = data.iloc[:,0].values

In [15]:
# Train Test Split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()

gnb.fit(x_train,y_train)

y_pred = gnb.predict(x_test)

print('The accuracy of the Random Forest is',metrics.accuracy_score(y_pred,y_test))

The accuracy of the Random Forest is 0.625
